<a href="https://colab.research.google.com/github/rick-phyto/cheminformatics/blob/main/get_80P40C_CID_Sm_indices_doPhenolcount_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive #new for csv
drive.mount('/content/gdrive') #new for csv


import pandas as pd

!pip install rdkit

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np

#import requests
#from bs4 import BeautifulSoup #no longer necessary since we aren't parsing xml
#import re

In [ ]:
#these are sorted lists with no duplicates - try a binary search from python recipes

def binarySearch(alist, item):
    first = 0
    last = len(alist)-1
    found = False

    while first<=last and not found:
        pos = 0
        midpoint = (first + last)//2
        if alist[midpoint] == item:
            pos = midpoint
            found = True
        else:
            if item < alist[midpoint]:
                last = midpoint-1
            else:
                first = midpoint+1
    return (pos, found)

In [ ]:
JasialIndices=pd.read_csv('gdrive/My Drive/Colab Notebooks/JasialToPubchem-indices.csv') #has a header field 'cid'
JasialIndices.head

In [ ]:
df_firstn_cids = pd.DataFrame()

#this seems to work really well and very memory efficiently for some reason =) (Only 1 GB used =) )
df_firstn_cids = pd.read_csv('gdrive/My Drive/Colab Notebooks/CID-SMILES', header = None, sep='\t', usecols=[0], nrows=31000000) #can change nrows to 31000000 ; save RAM
df_firstn_cids.shape

(31000000, 1)

In [ ]:
pubchem_activecids_list = df_firstn_cids[0].tolist() #at 4.2GB RAM by end of this line #2.4GB when using 37M nrows
len(pubchem_activecids_list)

31000000

In [ ]:
#get the indices of pubchem_activecids with values matching JasialIndices
#for cid_iterator in JasialIndices
JasialIndices_list = JasialIndices['cid'].values.tolist()

In [ ]:
df_firstn_smiles = pd.read_csv('gdrive/My Drive/Colab Notebooks/CID-SMILES', header = None, sep='\t', usecols=[1], nrows=31000000)

In [ ]:
df_firstn_smiles[1].iloc[3]

'CC(CN)O'

In [ ]:
#wow - a million takes just six seconds to run.
relevant_cid_smiles_indices = []
Number_of_phenols_list = []
JasialIndices_activecid_list = []

#organic subset heavy atoms in SMILES
boron = 'B'; nitrogen = 'N'; phophorus = 'P'; sulfur = 'S';
fluorine = 'F'; chlorine = 'Cl'; bromine = 'Br'; iodine = 'I';
iron = '[Fe]'; zinc = '[Zn]'; potassium = '[K]'; calcium = '[Ca]';
magnesium = '[Mg]'; sodium = '[Na]'; lithium = '[Li]';


for iterator in JasialIndices_list:
  outtest = binarySearch(pubchem_activecids_list,iterator)
  if(outtest[1]):
    relevant_cid_smiles_indices.append(outtest[0])

    smiles_test = df_firstn_smiles[1].iloc[outtest[0]]
    
    #implement non-CHO atom filter here. 
    #code for filtering non-CHO atoms
    if (boron in smiles_test) or (nitrogen in smiles_test) or (phosphorus in smiles_test) \
    or (sulfur in smiles_test) or (fluorine in smiles_test) or (chlorine in smiles_test) \
    or (bromine in smiles_test) or (iodine in smiles_test) or (iron in smiles_test) \
    or (zinc in smiles_test) or (potassium in smiles_test) or (calcium in smiles_test) \
    or (magnesium in smiles_test) or (sodium in smiles_test) or (lithium in smiles_test):
      #do nothing
      pass 
    else:
      #continue to phenolcounting

      chemical_mol = Chem.MolFromSmiles(smiles_test)   #smiles)

      Num_phenols = Chem.Fragments.fr_phenol(chemical_mol)
      Number_of_phenols_list.append(Num_phenols)

      JasialIndices_activecid_list.append(iterator) #save only cids present in the current (2023) CID-SMILES dataset
    #else:
     #relevant_cid_smiles_indices.append(None)

#~1 minute 43 s of execution
#~10 s of execution if heavy atom filters applied.


len(Number_of_phenols_list) 


5521

In [ ]:
len(JasialIndices_activecid_list)
#now save Jasial CID numbers & phenolcount results side by side in a DataFrame, and write to a csv. (Should we also save SMILES string col?)
#was 5521 without expanded atom filter list Also 5521 with expanded atom filter list.
new_columns = ['CID', 'PhenolCount', 'SMILES']
df_phenolcount_cid_result = pd.DataFrame(columns = new_columns)
df_phenolcount_cid_result['PhenolCount'] = Number_of_phenols_list
df_phenolcount_cid_result['CID'] = JasialIndices_activecid_list
df_phenolcount_cid_result.head


In [ ]:
df_phenolcount_cid_result.to_csv('gdrive/My Drive/Colab Notebooks/cids_phenolcount.csv', index = False)
#also created _noOrtho_Hbond file

In [ ]:
#certain heavy atom detect exercise
boron = 'B'; nitrogen = 'N'; phosphorus = 'P'; sulfur = 'S';
fluorine = 'F'; chlorine = 'Cl'; bromine = 'Br'; iodine = 'I'

smiles_test = 'CC1C(C(C(C(O1)OClC2C(C(C(C(O2)OC3=CC(=C4C(=O)CC(OC4=C3)C5=CC(=C(C=C5)OC)O)O)O)O)O)O)O)O'

if (boron in smiles_test) or (nitrogen in smiles_test) or (phosphorus in smiles_test) \
    or (sulfur in smiles_test) or (fluorine in smiles_test) or (chlorine in smiles_test) \
    or (bromine in smiles_test) or (iodine in smiles_test):
  print("present")
else:
  print("not present")

present


Total active CIDs in CID_SMILES from JasialIndices is 330991 of 345207, or 95.9%.   

Following is relevant to this point onward only:
Using 'None' to fill in the missing places. Need this to result in a blank cell in the csv - like " ,, " (two consecutive commas).

In [ ]:
pd_phenols = pd.DataFrame(Number_of_phenols)
JasialIndices_list

In [ ]:
 df_csv_prep = pd.DataFrame(relevant_cid_smiles_indices)

In [ ]:
#becomes float here
df_csv_prep.head

In [ ]:
#and write that list of indices into a fresh csv. (not the greatest filename chosen here I'll grant)
df_csv_prep.to_csv('gdrive/My Drive/Colab Notebooks/relevant_cid_smiles_indices_noblanks.csv', sep=",", header=False)